In [21]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

# path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_set'
path = '/home/maroco/dataset/spam_dataset'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

# Create a list to store DataFrames
all_dfs = []

for file in filenames:
  try:
    # Use 'on_bad_lines' instead of 'error_bad_lines'
    # df = pd.read_csv(file, on_bad_lines='skip', delimiter=",")  # Skip bad lines
    df = pd.read_csv(file)

    # 정상화
    df.columns=['index', 'data']
    df = df.set_index('index')
    df['spam'] = 1
    df['data'] = df['data'].str.replace('ifg@','')

    # Dask Dataframe 생성
    # Move ddf creation after modifying the Pandas DataFrame
    ddf = dd.from_pandas(df, npartitions=4) # Create ddf after modifications


    # Append the current DataFrame to the list
    result = ddf.compute()
    all_dfs.append(result)

  except pd.errors.ParserError as e:
    print(f"Error reading file {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_dfs, ignore_index=True)

In [22]:
from konlpy.tag import Okt
import re

# 형태소 분석기 초기화
okt = Okt()

def preprocess_text(text):
    """
    텍스트를 전처리하고 주요 형태소만 남깁니다.
    """
    # 특수문자 제거
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)
    # 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    # 형태소 분석 후 명사와 동사만 추출
    tokens = okt.pos(text, stem=True)
    filtered_tokens = [word for word, pos in tokens if pos in ['Noun', 'Verb']]
    # 복원된 텍스트 생성
    return " ".join(filtered_tokens)

# 데이터셋 전처리
raw['data'] = raw['data'].apply(preprocess_text)

In [23]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['data'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

286598
286598


In [24]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('data')
print(len(raw))

286598
212618


In [25]:
raw['data'] = raw['data'].str.replace('Web발신', '').str.strip()
raw['data'] = raw['data'].str.replace('국제발신', '').str.strip()
raw['data'] = raw['data'].str.replace('광고', '').str.strip()
raw['data'] = raw['data'].str.replace('발신', '').str.strip()

In [26]:
raw.head()

,data,spam
0,종근당 건강 아이 크다 상자 드리다 아이 키 성장 도움 주다 주목 국내 유일 키 성...,1
1,신봉 대리점 여름 찾아오다 항상 기원 하다 마음 저희 신봉 점 이용 해주다 벤트 사...,1
2,김성춘 칼텍스 원 주유 쿠폰 무료 증정 선착순 마감 임박 또 나 모르다 뻔 하다 신...,1
3,농축 산화수 행사 우국 거리 한우 양념 불고기 무료 거부,1
4,레디 캐시 천원 선물 드리다 보다 문자 대명 임 레디 고객 발송 되다 정서 고객 이...,1


In [27]:
raw.to_csv('/home/maroco/dataset/preprocessed_set/spam_data.csv', index=False)